In [1]:
import json
import pandas as pd
from src.utils.UsefulPaths import Paths
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

2023-09-01 22:23:27,900 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-09-01 22:23:28,029 - DEBUG - Creating converter from 7 to 5
2023-09-01 22:23:28,029 - DEBUG - Creating converter from 5 to 7
2023-09-01 22:23:28,029 - DEBUG - Creating converter from 7 to 5
2023-09-01 22:23:28,029 - DEBUG - Creating converter from 5 to 7


In [2]:
paths = Paths()

with open(paths.json_subsectors, 'r') as file:
    subsectors = json.load(file)

In [3]:
df_ipc_hypothesis = pd.read_csv(paths.ipc_subsetor_hypothesis)
df_ipc_hypothesis.head()

,IPC,title,zero_shot_hypothesis,similarities_hypothesis
0,G16Y,INFORMATION AND COMMUNICATION TECHNOLOGY SPECI...,"Industry 4.0 , Other, Fintech, Edtech, Artific...","Other, Industry 4.0 , Agtech, Artificial Intel..."
1,F24J,HEATING; RANGES; VENTILATING,"Medtech / Medical devices, Industry 4.0 , Biop...","Other, Medtech / Medical devices, Biopharma / ..."
2,F21H,INCANDESCENT MANTLES; OTHER INCANDESCENT BODIE...,"Other, Medtech / Medical devices, Biopharma / ...","Other, Edtech, Cleantech, Medtech / Medical de..."
3,G16B,"BIOINFORMATICS, i.e. INFORMATION AND COMMUNICA...","Biopharma / Biotech, Artificial Intelligence, ...","Other, Biopharma / Biotech, Agtech, Artificial..."
4,A01P,"BIOCIDAL, PEST REPELLANT, PEST ATTRACTANT OR P...","Biopharma / Biotech, Other, Industry 4.0 , Med...","Other, Agtech, Biopharma / Biotech, New Food, ..."


In [4]:
ipc_dict = df_ipc_hypothesis.set_index('IPC').to_dict(orient='index')
for k, v in ipc_dict.items():
    ipc_dict[k]['zero_shot_hypothesis'] = ipc_dict[k]['zero_shot_hypothesis'].split(', ')
    ipc_dict[k]['similarities_hypothesis'] = ipc_dict[k]['similarities_hypothesis'].split(', ')

In [5]:
# ABSTRACT BY JOAO
df_abstract = pd.read_parquet('C:\\Users\\thiag\\PycharmProjects\\genome\\data\\processed\\abstract_and_classes.parquet')
df_abstract.dropna(inplace=True)
df_abstract.reset_index(drop=True, inplace=True)
df_abstract

,patent_id,abstract,class_concat,class_names
0,20180274840,A refrigerator comprises: a refrigerator compa...,F25D,Mechanical Engineering; Lighting; Heating; Wea...
1,20190097688,Embodiments of methods and systems for operati...,H04B,Electricity
2,20190157921,Provided is a magnetic plate laminate comprisi...,H02K,Electricity
3,20190189823,The present disclosure relates to a solar cell...,H01L,Electricity
4,20190220574,A system and protocol for integrating a plural...,G06F,Physics
...,...,...,...,...
1073256,20200412939,The invention relates to a device for measurin...,"G06T,G02B,H04N","Physics,Physics,Electricity"
1073257,20200413013,A system and method for altering the visual ap...,H04N,Electricity
1073258,20200413084,An apparatus for video decoding includes proce...,H04N,Electricity
1073259,20200413347,"Methods, systems, and devices for wireless com...","H04L,H04W","Electricity,Electricity"


In [6]:
# df_abstract = pd.read_parquet(paths.raw_parquet_abstract)
# df_abstract.head()

In [7]:
# offset_div = 50
# df_joao = df_abstract.iloc[:offset_div].copy().reset_index(drop='index')
# df_leo = df_abstract.iloc[offset_div:offset_div*2].copy().reset_index(drop='index')
# df_maxm = df_abstract.iloc[offset_div*2:offset_div*3].copy().reset_index(drop='index')
# df_rafael = df_abstract.iloc[offset_div*4:offset_div*5].copy().reset_index(drop='index')
# df_thiago = df_abstract.iloc[offset_div*6:offset_div*7].copy().reset_index(drop='index')
# df_thiago_many = df_abstract.iloc[offset_div*6:offset_div*32].copy().reset_index(drop='index')

In [10]:
all_subsetor_names = []
all_subsetor_definitions = []
all_subsetor_keywords = []
all_subsetor_does_include = []
all_subsetor_does_not_include = []
for subsetor_name, subsetor_values in subsectors.items():
    subsetor_definition = subsetor_values.get('Definition', '')
    subsetor_keywords = subsetor_values.get('Keywords', '')
    subsetor_does_include = subsetor_values.get('Does include', '')
    subsetor_does_not_include = subsetor_values.get('Does not include', '')

    all_subsetor_names.append(subsetor_name)
    all_subsetor_definitions.append(subsetor_definition)
    all_subsetor_keywords.append(subsetor_keywords)
    all_subsetor_does_include.append(subsetor_does_include)
    all_subsetor_does_not_include.append(subsetor_does_not_include)

all_subsetor_names.append('Other')
all_subsetor_definitions.append('')
all_subsetor_keywords.append('')
all_subsetor_does_include.append('')
all_subsetor_does_not_include.append('')

In [15]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# 'bert-base-uncased'
# 'bert-large-uncased-whole-word-masking'
# 'bert-base-multilingual-cased'
# 'all-mpnet-base-v2'
model = SentenceTransformer('all-mpnet-base-v2')
subsetor_definitions_embeddings = model.encode(all_subsetor_definitions, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=False)

df_all = pd.DataFrame()
for index, row in df_abstract.iterrows():
    patent_abstract = row['abstract']

    query = f'What is the subsetor definiton that best describes this patent abstract "{patent_abstract}"'
    patent_abstract_embeddings = model.encode(query, convert_to_tensor=True, show_progress_bar=False, normalize_embeddings=False)

    cos_scores = util.cos_sim(patent_abstract_embeddings, subsetor_definitions_embeddings)

    p = pd.DataFrame(cos_scores.cpu().numpy().tolist(), columns=all_subsetor_names)
    patent_similarities_hypothesis = p.iloc[0, :].to_frame().T.sort_values(by=0, axis=1, ascending=False).columns.tolist()

    patent_zero_shot_class = classifier(patent_abstract, all_subsetor_names, multi_label=False)

    p2 = pd.DataFrame([patent_zero_shot_class['scores']], columns=patent_zero_shot_class['labels'])
    patent_zero_shot_hypothesis = ', '.join(patent_zero_shot_class['labels'])

    patent_ipcs = row['class_concat'].split(', ')
    all_hypothesis = []
    for patent_ipc in patent_ipcs:
        ipc_zero_shot_hypothesis = ipc_dict[patent_ipc].get('zero_shot_hypothesis')
        ipc_similarities_hypothesis = ipc_dict[patent_ipc].get('zero_shot_hypothesis')

        candidate_zero_shot_label = ipc_zero_shot_hypothesis[0:5]
        candidate_similarities_label = ipc_similarities_hypothesis[0:5]

        output_zero_shot = classifier(patent_abstract, candidate_zero_shot_label, multi_label=False)
        all_hypothesis.append(output_zero_shot['labels'][0])

        output_similarities = classifier(patent_abstract, candidate_similarities_label, multi_label=False)
        all_hypothesis.append(output_similarities['labels'][0])

        a = 0

2023-09-01 22:40:33,271 - DEBUG - Resetting dropped connection: huggingface.co
2023-09-01 22:40:33,543 - DEBUG - https://huggingface.co:443 "HEAD /MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/resolve/main/config.json HTTP/1.1" 200 0
C:\Users\thiag\PycharmProjects\Genome\venv\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
2023-09-01 22:40:35,002 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-09-01 22:40:35,479 - INFO - Use pytorch device: cpu


KeyboardInterrupt: 